In [3]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
%matplotlib inline

### 표본추출

#### 단순랜덤추출법

In [16]:
data = load_iris()
iris_cols = list(data['feature_names']) + ['target']
iris = pd.DataFrame(np.c_[data['data'], data['target']], columns=
                    [col.replace(' (cm)', '') for col in iris_cols])
print(iris.head(3))

   sepal length  sepal width  petal length  petal width  target
0           5.1          3.5           1.4          0.2     0.0
1           4.9          3.0           1.4          0.2     0.0
2           4.7          3.2           1.3          0.2     0.0


In [17]:
data_list = [1,2,3,4,5,'a','b','c']
random.sample(data_list, 4)

[3, 1, 'a', 4]

In [18]:
# 넘파이 라이브러리
np.random.choice(data_list, 4, replace=True)

array(['a', 'c', 'a', '2'], dtype='<U11')

In [19]:
# 특정 범위 내의 정수/실수의 난수 생성
print('0~10 사이의 정수 중 3개의 난수 생성: ', np.random.randint(0, 10, 3))
print('0~1 사이의 실수를 2*2 배열로 생성:\n', np.random.rand(2,2))

0~10 사이의 정수 중 3개의 난수 생성:  [4 8 6]
0~1 사이의 실수를 2*2 배열로 생성:
 [[0.49208893 0.68524872]
 [0.83561078 0.18670298]]


In [20]:
print(iris.sample(n=3, replace=False))

     sepal length  sepal width  petal length  petal width  target
114           5.8          2.8           5.1          2.4     2.0
106           4.9          2.5           4.5          1.7     2.0
142           5.8          2.7           5.1          1.9     2.0


In [21]:
# frac 파라미터로 전체 데이터의 3%를 랜덤으로 추출할 수 있음.
print(iris.sample(frac=0.03))

     sepal length  sepal width  petal length  petal width  target
82            5.8          2.7           3.9          1.2     1.0
144           6.7          3.3           5.7          2.5     2.0
81            5.5          2.4           3.7          1.0     1.0
2             4.7          3.2           1.3          0.2     0.0


In [22]:
print(iris.sample(frac=0.03, weights='sepal length'))

     sepal length  sepal width  petal length  petal width  target
6             4.6          3.4           1.4          0.3     0.0
15            5.7          4.4           1.5          0.4     0.0
7             5.0          3.4           1.5          0.2     0.0
145           6.7          3.0           5.2          2.3     2.0


In [23]:
print(iris.sample(3, axis=1).head(3))

   petal length  petal width  sepal length
0           1.4          0.2           5.1
1           1.4          0.2           4.9
2           1.3          0.2           4.7


- 계통추출법

In [25]:
def sys_sampling(data, n) : 
    N = len(data)
    K = N//n
    index = data[:K].sample(1).index  # 첫 구간에서 임의로 선택한 샘플 1개의 인덱스
    # index개씩 띄어서 각 구간에서 하나씩 샘플 추출
    sys_df = pd.DataFrame()
    while len(sys_df) < n : 
        # sys_df = sys_df.append(data.loc[index,:])
        sys_df = pd.concat([sys_df, data.loc[index,:]])
        index += K
    return sys_df

print(sys_sampling(iris, 8))

     sepal length  sepal width  petal length  petal width  target
2             4.7          3.2           1.3          0.2     0.0
20            5.4          3.4           1.7          0.2     0.0
38            4.4          3.0           1.3          0.2     0.0
56            6.3          3.3           4.7          1.6     1.0
74            6.4          2.9           4.3          1.3     1.0
92            5.8          2.6           4.0          1.2     1.0
110           6.5          3.2           5.1          2.0     2.0
128           6.4          2.8           5.6          2.1     2.0


- 집락 추출법/층화추출법

In [34]:
def start_random_sampling(data, stratum, sampling_no, proportion=True) :
    if proportion == True : # 비례층화추출법: 원본 데이터 개수의 비율대로 추출
        levels = data[stratum].unique()
        total = data[stratum].value_counts().sum()
        prop_val = data[stratum].value_counts()/total
        no = prop_val * sampling_no
        result = pd.DataFrame()
        for level in levels : 
            temp_df = data[data[stratum]==level].sample(int(no[level]))
            result = pd.concat([result, temp_df])

    else :  # 불비례층화추출법: 임의로 정한 특정 비율대로 샘플링
        levels = list(proportion.keys())
        prop_val = np.array(list(proportion.values()))
        total = sum(prop_val)
        no = prop_val * sampling_no
        if total != 1 : 
            raise Exception('proportion sum is supposed to be 1.')
        else : 
            result = pd.DataFrame()
            for level in levels : 
                temp_df = data[data[stratum]==level].sample(int(no[level]))
                result = pd.concat([result, temp_df])

    return result

In [35]:
# 원본 층별 데이터 개수 확인: 층별로 동일하게 50개씩 관측값을 가지는 데이터
iris['target'].value_counts()

0.0    50
1.0    50
2.0    50
Name: target, dtype: int64

In [36]:
# 비례층화추출법으로 9개 샘플링하기: 원본과 동일한 비율로 샘플링하기
print(start_random_sampling(iris, 'target', 9))

     sepal length  sepal width  petal length  petal width  target
38            4.4          3.0           1.3          0.2     0.0
37            4.9          3.6           1.4          0.1     0.0
4             5.0          3.6           1.4          0.2     0.0
99            5.7          2.8           4.1          1.3     1.0
58            6.6          2.9           4.6          1.3     1.0
73            6.1          2.8           4.7          1.2     1.0
139           6.9          3.1           5.4          2.1     2.0
135           7.7          3.0           6.1          2.3     2.0
100           6.3          3.3           6.0          2.5     2.0


In [37]:
# 불비례층화추출법으로 10개 샘플링하기: 임의로 정한 비율로 샘플링하기
print(start_random_sampling(iris, 'target', 10, proportion={0:0.2, 1:0.5,
                                                            2:0.3}))

     sepal length  sepal width  petal length  petal width  target
23            5.1          3.3           1.7          0.5     0.0
15            5.7          4.4           1.5          0.4     0.0
71            6.1          2.8           4.0          1.3     1.0
74            6.4          2.9           4.3          1.3     1.0
99            5.7          2.8           4.1          1.3     1.0
53            5.5          2.3           4.0          1.3     1.0
51            6.4          3.2           4.5          1.5     1.0
127           6.1          3.0           4.9          1.8     2.0
102           7.1          3.0           5.9          2.1     2.0
121           5.6          2.8           4.9          2.0     2.0
